# Mine Safety

We're interested in [US mine safety](https://arlweb.msha.gov/drs/drshome.htm), thank goodness we can search for these things.

## Setup: Import what you'll need to search and scrape and Selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd

import requests
from bs4 import BeautifulSoup

## Starting from `https://arlweb.msha.gov/drs/drshome.htm`, search for every operator with 'dirt' in their name, including abandoned mines.

> - *Tip: If you can't make an element work using name, class or ID, try to use the XPath*

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get('https://arlweb.msha.gov/drs/drshome.htm')

In [4]:
text_input = driver.find_element_by_name('OperSearch')

In [5]:
text_input.send_keys('dirt')

In [6]:
abandoned_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table/tbody/tr[3]/td[3]/table/tbody/tr/td/input')
abandoned_button.click()

In [7]:
search = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table/tbody/tr[7]/td[3]/input[1]')
search.click()

## Scrape the results page, saving it as `dirt-operators.csv`

> - *Tip: Think about what each row in your dataset will be, and start by looping through that*
> - *Tip: Printing is cool and good! Print everything! Move it into a dictionary later.*
> - *Tip: If you don't want a row, think about what's in the row that makes it different. You can use an `if` statement or list slicing to skip the ones you aren't interested in.*
> - *Tip: Make sure your dictionary and your loop variable have DIFFERENT NAMES*
> - *Tip: After you've made your dictionary (and printed it, of course), you'll want to add it to your list of rows*
> - *Tip: Be sure to import pandas to convert it to a dataframe*
> - *Tip: Make sure you don't include the index when saving your dataframe*

### Hopefully you know that each `tr` is supposed to be a row of your data. What is the index of the first row element that is actually a result?

> - *Tip: `.text` will help you here.*
> - *Tip: You aren't interesting in annotations or anything, just mines and where they are from*
> - *Tip: Using `print("-----")` will help you keep track of different rows*
> - *Tip: If you have a list called `animals`, `animals[2:]` will skip the first two and start with the third. You can use this to skip ahead to the 'good' data if you want*

In [8]:
driver.find_elements_by_tag_name('tr')[7].text
        



'3503598\nOR  Newberg Rock & Dirt   Newberg Rock & Dirt Surface M  Active  Crushed, Broken Stone NEC '

### Loop through each operator result, printing its name

> - *Tip: If you have a list called `animals`, `animals[2:]` will skip the first two and start with the third.*
> - *Tip: You can use list slicing or an `if` statement to skip the non-data row(s). List slicing is probably easier, even if you aren't comfortable with it.*
> - *Tip: or honestly you can use `try` and `except` if you know how it works.*
> - *Tip: Once you have the "right" rows of data, you're going to be looking for a certain tag inside*
> - *Tip: Sometimes you can't say "give me this class," and instead you have to say "give me all of the `div` elements, and then give me the third one."*

In [9]:
for rows in driver.find_elements_by_tag_name('tr')[7:]:
    print("--------")
    Operator = rows.find_elements_by_tag_name('td')[2].text
    print(Operator)

--------
Newberg Rock & Dirt  
--------
Allied Dirt Moving Company  
--------
AM Dirtworks & Aggregate Sales  
--------
Atlas-Dirty Devil Mining  
--------
Atlas-Dirty Devil Mining  
--------
Babe's Dirt Work  
--------
Bar-Lin Dirt Company  
--------
Barber'S Dirt Pit  
--------
Bender Sand & Dirt  
--------
BERT'S DIRT  
--------
Big D Dirt Service Inc  
--------
Big Red Dirt Farm LLC  
--------
Big River Dirt Pit  
--------
Bob Harris Dirt Contracting  
--------
Bohannon Sand & Dirt  
--------
Bratcher'S Sand & Dirt  
--------
Brewer Dirt Works  
--------
Buck'S Dirt Pit  
--------
C & G Dirt Hauling  
--------
C N C Dirt Movers, Inc.  
--------
Cambridge Dirt Sand and Gravel LLC  
--------
Central Iowa Dirt & Demo LLC  
--------
Crowes Trucking & Dirt Pit Services  
--------
D & H Dirt  
--------
Diez Dirt & Sand Hauling Inc  
--------
Dirt Cheap  
--------
Dirt Company  
--------
Dirt Company  
--------
Dirt Company  
--------
Dirt Con  
--------
Dirt Diggers Inc  
--------
Dirt D

IndexError: list index out of range

### Loop through each operator result, printing its ID

There should be ONE code per row, and NO empty rows between them.

In [11]:
for rows in driver.find_elements_by_tag_name('tr')[7:]:
    print("--------")
    ID = rows.find_elements_by_tag_name('td')[0].text
    print(ID)

--------
3503598
--------
0502030
--------
4801789
--------
4201449
--------
4201450
--------
1002257
--------
1601167
--------
4103265
--------
1401575
--------
1700776
--------
1601251
--------
0301963
--------
1601082
--------
3401751
--------
1600916
--------
3401211
--------
0301267
--------
1600956
--------
2200033
--------
0504953
--------
3401929
--------
1302445
--------
1601106
--------
3400915
--------
1600983
--------
4503200
--------
3401266
--------
3401468
--------
5001797
--------
4608254
--------
1510279
--------
2103723
--------
0100776
--------
4104016
--------
2103914
--------
4104757
--------
0301729
--------
0404851
--------
2200734
--------
5002028
--------
1513393
--------
3800602
--------
3101630
--------
3200860
--------
3401762
--------
2103517
--------
2402626
--------
2103181
--------
1601124
--------
1601150
--------
4703427
--------
0801306
--------
2501216
--------
3200965
--------
2901371
--------
2901544
--------
2901709
--------
4102355
--------
41024

## Saving the results

### Loop through each `tr` to create a list of dictionaries

Each dictionary must contain

- Operator ID
- Operator name
- Mine name
- State
- Mine type
- Coal or metal
- Status
- Commodity

Create a new dictionary for each row.

> - *Tip: Start with an empty dictionary, then add the keys one at a time like we did during class*
> - *Tip: You might want to save all of the cells in a variable, then use indexes to get the second, third, fourth, etc.*
> - *Tip: I know you already skipped a bunch of rows already, but one of them still might be bad! Which one is it? How can you skip it? You might need to slice out some of the end of your list, too. Use `print` to help you debug, or just look at the page closely.*
> - *Tip: Or, if you did the other homework already, `try` / `except` is also an option*

In [12]:
mines_list = driver.find_elements_by_tag_name('tr')[7:-1]

In [13]:
rows = []

for mine in mines_list:
    
    row = {}
    columns = mine.find_elements_by_tag_name('td')

    ID = columns[0]
    row['ID'] = ID.text
    
    State =  columns[1]
    row['State'] = State.text
    
    Operator = columns[2]
    row['Operator'] = Operator.text
    
    Mine_Name = columns[3]
    row['Mine_Name'] = Mine_Name.text
    
    Type = columns[4]
    row['Type'] = Type.text
    
    CM = columns[5]
    row['CM'] = CM.text
    
    Status = columns[6]
    row['Status'] = Status.text
    
    Commodity = columns[7]
    row['Commodity'] = Commodity.text
    
    print("dictionary look like" , row['State'])
    
    rows.append(row)

    

dictionary look like OR 
dictionary look like CO 
dictionary look like ND 
dictionary look like UT 
dictionary look like UT 
dictionary look like ID 
dictionary look like LA 
dictionary look like TX 
dictionary look like KS 
dictionary look like ME 
dictionary look like LA 
dictionary look like AR 
dictionary look like LA 
dictionary look like OK 
dictionary look like LA 
dictionary look like OK 
dictionary look like AR 
dictionary look like LA 
dictionary look like MS 
dictionary look like NM 
dictionary look like OK 
dictionary look like IA 
dictionary look like LA 
dictionary look like OK 
dictionary look like LA 
dictionary look like WA 
dictionary look like OK 
dictionary look like OK 
dictionary look like AK 
dictionary look like WV 
dictionary look like KY 
dictionary look like MN 
dictionary look like AL 
dictionary look like TX 
dictionary look like MN 
dictionary look like TX 
dictionary look like AR 
dictionary look like CA 
dictionary look like MS 
dictionary look like AK 


In [14]:
import pandas as pd

df = pd.DataFrame(rows)

df

,CM,Commodity,ID,Mine_Name,Operator,State,Status,Type
0,M,"Crushed, Broken Stone NEC",3503598,Newberg Rock & Dirt,Newberg Rock & Dirt,OR,Active,Surface
1,M,Construction Sand and Gravel,0502030,Allied Dirt Moving Co Pit & Plant,Allied Dirt Moving Company,CO,Abandoned,Surface
2,M,Construction Sand and Gravel,4801789,AM Dirtworks & Aggregate Sales,AM Dirtworks & Aggregate Sales,ND,Abandoned,Surface
3,C,Coal (Bituminous),4201449,Unit Train Loading Facility,Atlas-Dirty Devil Mining,UT,Abandoned,Facility
4,C,Coal (Bituminous),4201450,Blackie Surface Mine & Prep Plant,Atlas-Dirty Devil Mining,UT,Abandoned,Surface
5,M,Construction Sand and Gravel,1002257,"Hitt Pit, Inc.",Babe's Dirt Work,ID,Abandoned,Surface
6,M,Construction Sand and Gravel,1601167,Bar-Lin Dirt Pit,Bar-Lin Dirt Company,LA,Abandoned,Surface
7,M,Construction Sand and Gravel,4103265,Barber'S Dirt Pit,Barber'S Dirt Pit,TX,Abandoned,Surface
8,M,Construction Sand and Gravel,1401575,BENDER SAND & DIRT,Bender Sand & Dirt,KS,Intermittent,Surface
9,M,Construction Sand and Gravel,1700776,BERT'S DIRT,BERT'S DIRT,ME,Abandoned,Surface


In [15]:
len(rows)

132

In [16]:
df.head(50)

,CM,Commodity,ID,Mine_Name,Operator,State,Status,Type
0,M,"Crushed, Broken Stone NEC",3503598,Newberg Rock & Dirt,Newberg Rock & Dirt,OR,Active,Surface
1,M,Construction Sand and Gravel,0502030,Allied Dirt Moving Co Pit & Plant,Allied Dirt Moving Company,CO,Abandoned,Surface
2,M,Construction Sand and Gravel,4801789,AM Dirtworks & Aggregate Sales,AM Dirtworks & Aggregate Sales,ND,Abandoned,Surface
3,C,Coal (Bituminous),4201449,Unit Train Loading Facility,Atlas-Dirty Devil Mining,UT,Abandoned,Facility
4,C,Coal (Bituminous),4201450,Blackie Surface Mine & Prep Plant,Atlas-Dirty Devil Mining,UT,Abandoned,Surface
5,M,Construction Sand and Gravel,1002257,"Hitt Pit, Inc.",Babe's Dirt Work,ID,Abandoned,Surface
6,M,Construction Sand and Gravel,1601167,Bar-Lin Dirt Pit,Bar-Lin Dirt Company,LA,Abandoned,Surface
7,M,Construction Sand and Gravel,4103265,Barber'S Dirt Pit,Barber'S Dirt Pit,TX,Abandoned,Surface
8,M,Construction Sand and Gravel,1401575,BENDER SAND & DIRT,Bender Sand & Dirt,KS,Intermittent,Surface
9,M,Construction Sand and Gravel,1700776,BERT'S DIRT,BERT'S DIRT,ME,Abandoned,Surface


In [ ]:
df.to_csv("dirt-operators.csv" , index=False)

In [17]:
pd.read_csv("dirt-operators.csv").head()

,CM,Commodity,ID,Mine_Name,Operator,State,Status,Type
0,M,"Crushed, Broken Stone NEC",3503598,Newberg Rock & Dirt,Newberg Rock & Dirt,OR,Active,Surface
1,M,Construction Sand and Gravel,502030,Allied Dirt Moving Co Pit & Plant,Allied Dirt Moving Company,CO,Abandoned,Surface
2,M,Construction Sand and Gravel,4801789,AM Dirtworks & Aggregate Sales,AM Dirtworks & Aggregate Sales,ND,Abandoned,Surface
3,C,Coal (Bituminous),4201449,Unit Train Loading Facility,Atlas-Dirty Devil Mining,UT,Abandoned,Facility
4,C,Coal (Bituminous),4201450,Blackie Surface Mine & Prep Plant,Atlas-Dirty Devil Mining,UT,Abandoned,Surface


### Save that to a CSV named `dirt-operators.csv`

In [ ]:
df.to_csv("dirt-operators.csv", index=False)

### Open the CSV file and examine the first few.

Make sure you didn't save that extra weird unnamed index column.